In [14]:
from sagemaker import get_execution_role
from sagemaker.session import Session
from sagemaker.tensorflow import TensorFlow

In [15]:
import boto3
import random
import json
import bidding_data
import config

In [16]:
bidding_data.download_stats_file_from_s3()

In [17]:
# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket here if you wish.
bucket = 'wsbidder'

# Location where results of model training are saved.
output_path = 's3://{}/trainer_predict_bid/model_output'.format(bucket)
model_dir = 's3://{}/trainer_predict_bid/model'.format(bucket)

# train_instance_type = 'ml.c4.xlarge'
train_instance_type = 'ml.p2.xlarge'
train_instance_count = 1
hyperparameters = {'epochs': 500, 'batch_size': 8, 'config_file': 'config.json'}
inputs = {'train': 's3://wsbidder/trainer_predict_bid/data/train/',
          'eval': 's3://wsbidder/trainer_predict_bid/data/eval/',
          'test': 's3://wsbidder/trainer_predict_bid/data/eval/'}
# inputs = {'train': '/opt/ml/input/data/train/',
#           'eval': '/opt/ml/input/data/eval/',
#           'test': '/opt/ml/input/data/eval/'}
base_job_name = 'tf-bid-prediction'

In [18]:
estimator = TensorFlow(entry_point='trainer.py',
                       source_dir='.',
                       output_path=output_path,
                       model_dir=model_dir,
                       train_instance_type=train_instance_type,
                       train_instance_count=train_instance_count,
                       hyperparameters=hyperparameters,
                       role=get_execution_role(),
#                        image_name='520713654638.dkr.ecr.eu-west-1.amazonaws.com/sagemaker-tensorflow-scriptmode:1.12.0-gpu-py3',
                       base_job_name=base_job_name,
                       framework_version='1.14.0',
                       py_version='py3',
#                        distributions={'parameter_server': {'enabled': True}},
                       script_mode=True)

In [19]:
# from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

# # Define objective
# objective_metric_name = 'loss'
# objective_type = 'Minimize'
# metric_definitions = [{'Name': 'loss',
#                        'Regex': 'loss = ([0-9\\.]+)'}]
# # Define hyperparameter ranges
# hyperparameter_ranges = {
#                             'learning_rate': ContinuousParameter(1e-6, 1e-4)
# #                             'dropout_rate': ContinuousParameter(0.0, 0.5)
#                         }  
# # hyperparameter_ranges = {
# #                             'learning_rate': ContinuousParameter(0.0000001, 0.0001),
# #                             'dropout_rate': ContinuousParameter(0.0, 0.9),
# #                             'batch_size': IntegerParameter(512, 4096)
# #                         }  
# # Initialise Sagemaker's hyperparametertuner
# tuner = HyperparameterTuner(estimator,
#                             objective_metric_name,
#                             hyperparameter_ranges,
#                             metric_definitions,
#                             max_jobs=4,
#                             max_parallel_jobs=2,
#                             objective_type=objective_type)

In [20]:
s3 = boto3.resource('s3')
for obj in s3.Bucket(bucket).objects.filter(Prefix='trainer_predict_bid/model/'):
    s3.Object(bucket,obj.key).delete()

In [21]:
# tuner.fit(inputs)

In [22]:
# tuner.deploy(
#     initial_instance_count=1,
#     instance_type='ml.c5.large',
#     endpoint_name='BidPredictionProductionEndpoint',
#     update_endpoint=True
# )

In [23]:
# from sagemaker.tensorflow.serving import Model

# model_data = 's3://wsbidder/trainer_predict_imp/model_output/sagemaker-tensorflow-190731-1106-015-063c2327/output/model.tar.gz'

# model = Model(
#     model_data=model_data, 
#     role=get_execution_role()
# )
# model.deploy(
#     initial_instance_count=1,
#     instance_type='ml.c5.large',
#     endpoint_name='BidPredictionProductionEndpoint',
#     update_endpoint=True
# )

In [24]:
%%time

estimator.fit(inputs)

2019-12-05 07:46:50 Starting - Starting the training job...
2019-12-05 07:46:51 Starting - Launching requested ML instances......
2019-12-05 07:48:16 Starting - Preparing the instances for training.........
2019-12-05 07:49:25 Downloading - Downloading input data...
2019-12-05 07:49:52 Training - Downloading the training image...
2019-12-05 07:50:50 Training - Training image download completed. Training in progress../usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8

W1205 07:50:59.596193 140360579442432 deprecation.py:323] From /opt/ml/code/bidding_data.py:75: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W1205 07:51:03.339416 140360579442432 deprecation.py:323] From /opt/ml/code/losses.py:11: Normal.__init__ (from tensorflow.python.ops.distributions.normal) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
W1205 07:51:03.342162 140360579442432 deprecation.py:323] From /usr/local/lib/python3.6/site-packages/tensorflow/python/ops/distributions/normal.py:160: Distribution.__init__ (from tensorflow.python.ops.distributions.distribution) is deprecated and will be removed after 2019-01-01.
Instructions for updating

I1205 07:52:00.833875 140360579442432 basic_session_run_hooks.py:606] Saving checkpoints for 300 into s3://wsbidder/trainer_predict_bid/model/1575532256/model.ckpt.
I1205 07:52:04.190626 140360579442432 estimator.py:1145] Calling model_fn.
I1205 07:52:04.190906 140360579442432 estimator.py:1145] Calling model_fn.
I1205 07:52:07.766943 140360579442432 estimator.py:1147] Done calling model_fn.
I1205 07:52:08.078670 140360579442432 estimator.py:1147] Done calling model_fn.
I1205 07:52:08.102421 140360579442432 evaluation.py:255] Starting evaluation at 2019-12-05T07:52:08Z
I1205 07:52:08.554992 140360579442432 monitored_session.py:240] Graph was finalized.
I1205 07:52:08.655678 140360579442432 saver.py:1286] Restoring parameters from s3://wsbidder/trainer_predict_bid/model/1575532256/model.ckpt-300
I1205 07:52:09.296612 140360579442432 session_manager.py:500] Running local_init_op.
I1205 07:52:09.357831 140360579442432 session_manager.py:502] Done running local_init_op.
I1205 07:52:11.0369

I1205 07:52:53.109057 140360579442432 builder_impl.py:421] SavedModel written to: s3://wsbidder/trainer_predict_bid/model/1575532256/export/predict_bid/temp-b'1575532366'/saved_model.pb
I1205 07:52:54.599410 140360579442432 estimator.py:368] Loss for final step: 0.34212524.
Finished training
I1205 07:52:54.773147 140360579442432 estimator.py:1145] Calling model_fn.
I1205 07:52:54.773410 140360579442432 estimator.py:1145] Calling model_fn.
I1205 07:52:57.642431 140360579442432 estimator.py:1147] Done calling model_fn.
I1205 07:52:57.642635 140360579442432 estimator.py:1147] Done calling model_fn.
I1205 07:52:57.643187 140360579442432 export_utils.py:170] Signatures INCLUDED in export for Classify: None
I1205 07:52:57.643320 140360579442432 export_utils.py:170] Signatures INCLUDED in export for Regress: ['serving_default', 'regression']
I1205 07:52:57.643444 140360579442432 export_utils.py:170] Signatures INCLUDED in export for Predict: ['predict']
I1205 07:52:57.643545 140360579442432 e

In [25]:
# estimator.deploy(
#     initial_instance_count=1,
#     instance_type='ml.c5.large',
#     endpoint_name='BidPredictionProductionEndpoint'
# #     update_endpoint=True
# )

In [26]:
# estimator.delete_endpoint()